In [5]:
import simpy
import numpy as np
import random
import pandas as pd
import networkx as nx

# Constants and Simulation Parameters
KB_TO_MB = 1 / 1024
BIT_TO_MBIT = 1 / 1024
MHZ_TO_HZ = 1e6
ENV_RUN_TIME = 1000  # simulation run time
LC = 1e-24  # power usage constant
Y_NT = 0.5  # Weighted factor for local time cost
Y_NE = 0.5  # Weighted factor for energy cost

# Networking and Computational Parameters
transmission_rate = [500, 1000, 1500, 2000, 2500, 3000]  # in kb/s
num_devices_list = [50, 100, 150, 200, 250, 300]
task_data_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45]  # task data in Mbit
max_power_consumption = 1000  # in Joules

# CPU and power parameters for devices and MEC server
local_device_cycle_freq = random.uniform(2, 4) * MHZ_TO_HZ  # in Hz
mec_server_cycle_freq = random.uniform(4, 8) * MHZ_TO_HZ  # in Hz
local_device_power = random.uniform(100, 400) / 1000  # in Watts
clock_cycles_per_bit = random.randint(800, 1200)

class MECServer:
    def __init__(self, env, capacity):
        self.env = env
        self.capacity = capacity
        self.processor = simpy.Resource(env, capacity=int(capacity))
        self.dag = nx.DiGraph()

    def add_task_to_dag(self, task_id, dependencies, task_data):
        self.dag.add_node(task_id, data=task_data, vk=random.random())
        for dep in dependencies:
            self.dag.add_edge(dep, task_id)

    def process_task(self, task_id):
        task_info = self.dag.nodes[task_id]
        data_bits = task_info['data'] * BIT_TO_MBIT * 1e6
        clock_cycles = data_bits * clock_cycles_per_bit

        local_time = clock_cycles / local_device_cycle_freq
        edge_time = clock_cycles / self.capacity
        total_task_time = (1 - task_info['vk']) * local_time + task_info['vk'] * edge_time
        
        yield self.env.timeout(total_task_time)
        return total_task_time

    def execute_tasks(self):
        for task in nx.topological_sort(self.dag):
            yield self.env.process(self.process_task(task))
            ct = self.env.now
            self.dag.nodes[task]['ct'] = ct

class Device:
    def __init__(self, env, mec_servers, logs, num_devices):
        self.env = env
        self.mec_servers = mec_servers
        self.logs = logs
        self.num_devices = num_devices

    def send_task(self, task_data, dependencies=[]):
        mec = min(self.mec_servers, key=lambda x: len(x.dag))
        task_id = self.env.now
        mec.add_task_to_dag(task_id, dependencies, task_data)

        # Calculate the local computation parameters
        Hn = task_data * BIT_TO_MBIT * 1e6  # Convert Mbit to bits
        Rn = clock_cycles_per_bit  # CPU cycles per bit
        Qn = random.uniform(0, 1)  # Offloading decision
        local_cycles = Hn * Rn * (1 - Qn)
        RTl = local_cycles / local_device_cycle_freq
        ATl = LC * local_device_cycle_freq**2 * local_cycles
        DTl = Y_NT * RTl + Y_NE * ATl

        selected_rate = random.choice(transmission_rate) * 1000  # Select a rate and convert kb/s to b/s
        transmission_time = (Hn * Qn) / selected_rate
        energy_consumed = transmission_time * local_device_power + ATl

        yield self.env.timeout(transmission_time + RTl)
        yield self.env.process(mec.execute_tasks())

        ct = mec.dag.nodes[task_id]['ct']
        total_energy = energy_consumed + (ct - self.env.now) * local_device_power

        self.logs.append({
            'num_devices': self.num_devices,
            'task_data': task_data,
            'completion_time': ct,
            'energy_consumed': total_energy,
            'weighted_waste': DTl,
            'transmission_rate': selected_rate / 1000  # Log rate in kb/s
        })

env = simpy.Environment()
mecs = [MECServer(env, mec_server_cycle_freq) for _ in num_devices_list]
logs = []

for num_devices in num_devices_list:
    devices = [Device(env, mecs, logs, num_devices) for _ in range(num_devices)]
    for device in devices:
        task_size = random.choice(task_data_sizes)
        env.process(device.send_task(task_size))

env.run(until=ENV_RUN_TIME)

df_logs = pd.DataFrame(logs)
print(df_logs.head())
if not df_logs.empty and df_logs['energy_consumed'].max() > max_power_consumption:
    print("Warning: Power consumption exceeded the maximum limit.")


   num_devices  task_data  completion_time  energy_consumed  weighted_waste  \
0          300          5         1.542899         0.001075        0.000430   
1          300          5         1.545032         0.000644        0.002149   
2          200          5         1.548682         0.001071        0.003327   
3           50         10         1.560368         0.001605        0.008363   
4          150          5         1.567311         0.000794        0.013062   

   transmission_rate  
0             1500.0  
1             2500.0  
2             1500.0  
3             2000.0  
4             2000.0  
